## 提取“公因数”曲线并导出


In [5]:
import os
from pathlib import Path

import lasio
import numpy as np


In [6]:
def extract_common_curves(input_folder, output_folder, base_curves=None):
    """
    从输入文件夹读取所有LAS文件，提取基础曲线+公共曲线，并保存到输出文件夹

    Args:
        input_folder: 输入LAS文件夹路径
        output_folder: 输出LAS文件夹路径
        base_curves: 基础必须曲线列表，会在此基础上再查找公共曲线
    """
    # 创建输出文件夹
    output_path = Path(output_folder)
    output_path.mkdir(parents=True, exist_ok=True)

    # 获取所有LAS文件
    input_path = Path(input_folder)
    # 修复：避免重复匹配，统一转换为小写处理
    las_files_lower = list(input_path.glob("*.las"))
    las_files_upper = list(input_path.glob("*.LAS"))

    # 合并并去重（使用文件路径的绝对路径去重）
    las_files = list(set(las_files_lower + las_files_upper))
    las_files.sort()  # 排序便于查看

    if not las_files:
        print(f"在 {input_folder} 中未找到LAS文件")
        return

    print(f"找到 {len(las_files)} 个LAS文件")

    # 第一步：找出所有文件的公共曲线
    all_common_curves = None
    for las_file in las_files:
        try:
            las = lasio.read(las_file, mnemonic_case="upper")
            file_curves = set([c.mnemonic for c in las.curves])

            if all_common_curves is None:
                all_common_curves = file_curves
            else:
                all_common_curves = all_common_curves.intersection(file_curves)
        except Exception as e:
            print(f"读取文件 {las_file.name} 时出错: {e}")
            continue

    if all_common_curves is None:
        print("无法读取任何LAS文件")
        return

    print(f"\n所有文件的公共曲线: {sorted(all_common_curves)}")

    # 第二步：合并基础曲线和公共曲线
    if base_curves:
        base_curves_set = set([c.upper() for c in base_curves])
        print(f"指定的基础曲线: {sorted(base_curves_set)}")

        # 找出基础曲线中不在公共曲线里的
        missing_in_common = base_curves_set - all_common_curves
        if missing_in_common:
            print(f"⚠️ 注意：以下基础曲线不是所有文件都有: {sorted(missing_in_common)}")

        # 找出公共曲线中不在基础曲线里的额外曲线
        extra_common = all_common_curves - base_curves_set
        if extra_common:
            print(f"✓ 发现额外的公共曲线: {sorted(extra_common)}")

        # 最终要提取的曲线 = 基础曲线（不管是否公共）+ 额外的公共曲线
        curves_to_extract = sorted(base_curves_set | all_common_curves)
    else:
        # 如果没有指定基础曲线，就只用公共曲线
        curves_to_extract = sorted(all_common_curves)

    print(f"\n最终将提取的曲线: {curves_to_extract}")
    print(f"共 {len(curves_to_extract)} 条曲线\n")
    print("=" * 60)

    # 第三步：处理每个LAS文件
    success_count = 0
    failed_files = []

    for las_file in las_files:
        try:
            # 读取LAS文件
            las = lasio.read(las_file, mnemonic_case="upper")

            # 获取文件中存在的曲线
            available_curves = [c.mnemonic for c in las.curves]

            # 找出实际可以提取的曲线
            actual_curves = [c for c in curves_to_extract if c in available_curves]
            missing_curves = [c for c in curves_to_extract if c not in available_curves]

            if missing_curves:
                print(f"⚠️ 文件 {las_file.name} 缺少曲线: {missing_curves}")

            if not actual_curves:
                print(f"✗ 文件 {las_file.name} 中没有找到任何指定的曲线，跳过")
                failed_files.append((las_file.name, "无指定曲线"))
                continue

            # 创建新的LAS对象
            new_las = lasio.LASFile()

            # 复制版本和井信息
            new_las.version = las.version
            new_las.well = las.well
            new_las.params = las.params
            new_las.other = las.other

            # 清空curves section
            new_las.curves = lasio.SectionItems()

            # 按照curves_to_extract的顺序添加曲线
            for curve_name in curves_to_extract:
                if curve_name in available_curves:
                    curve = las.curves[curve_name]
                    new_las.append_curve(
                        mnemonic=curve.mnemonic, data=curve.data, unit=curve.unit, descr=curve.descr, value=curve.value
                    )

            # 保存新的LAS文件
            output_file = output_path / las_file.name
            new_las.write(str(output_file), version=2.0)

            print(f"✓ 成功处理 {las_file.name}，提取了 {len(actual_curves)}/{len(curves_to_extract)} 条曲线")
            success_count += 1

        except Exception as e:
            print(f"✗ 处理文件 {las_file.name} 时出错: {e}")
            failed_files.append((las_file.name, str(e)))

    # 输出统计信息
    print("\n" + "=" * 60)
    print(f"处理完成！")
    print(f"成功: {success_count}/{len(las_files)}")

    if failed_files:
        print(f"\n失败的文件:")
        for filename, error in failed_files:
            print(f"  - {filename}: {error}")

In [7]:
# 定义输入输出路径
input_folder = "../data/vertical_well_las"
output_folder = "../data/vertical_well_common_las"

# 定义基础必须曲线
base_curves = ["DEPT", "GR", "DEN", "SP", "CAL"]

# 执行提取
extract_common_curves(input_folder, output_folder, base_curves)

找到 7 个LAS文件

所有文件的公共曲线: ['AMP', 'BOOL_POR', 'CN', 'DEN', 'DEPT', 'DT', 'FACIES', 'FLUIDS', 'GR', 'GR-NORM', 'GR1', 'GRINPEFA', 'INPEFA', 'LITH', 'LITH_SHOW', 'LLD1', 'MLP-YC', 'PEFA', 'PERM', 'POR', 'SAND??SHADIBI', 'SESMIC', 'SESMIC2', 'SVM-YC', 'SW', 'TWTPICKED', 'TWTPICKED2', 'VSH']
指定的基础曲线: ['CAL', 'DEN', 'DEPT', 'GR', 'SP']
⚠️ 注意：以下基础曲线不是所有文件都有: ['CAL', 'SP']
✓ 发现额外的公共曲线: ['AMP', 'BOOL_POR', 'CN', 'DT', 'FACIES', 'FLUIDS', 'GR-NORM', 'GR1', 'GRINPEFA', 'INPEFA', 'LITH', 'LITH_SHOW', 'LLD1', 'MLP-YC', 'PEFA', 'PERM', 'POR', 'SAND??SHADIBI', 'SESMIC', 'SESMIC2', 'SVM-YC', 'SW', 'TWTPICKED', 'TWTPICKED2', 'VSH']

最终将提取的曲线: ['AMP', 'BOOL_POR', 'CAL', 'CN', 'DEN', 'DEPT', 'DT', 'FACIES', 'FLUIDS', 'GR', 'GR-NORM', 'GR1', 'GRINPEFA', 'INPEFA', 'LITH', 'LITH_SHOW', 'LLD1', 'MLP-YC', 'PEFA', 'PERM', 'POR', 'SAND??SHADIBI', 'SESMIC', 'SESMIC2', 'SP', 'SVM-YC', 'SW', 'TWTPICKED', 'TWTPICKED2', 'VSH']
共 30 条曲线

✓ 成功处理 PH1.las，提取了 30/30 条曲线
✓ 成功处理 PH13.las，提取了 30/30 条曲线
✓ 成功处理 PH2.las，提取了 30/